In [ ]:
#@title Load packages and data

import torch as th
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sktime.datasets import load_gunpoint
from torch.utils.data import Dataset, DataLoader
from sklearn.neighbors import KNeighborsClassifier
import wandb
wandb.init(
    project="ssl-ourECG",
    config={
    "learning_rate": 0.0003,
    "architecture": "resnet18to34+fcn",  #  
    "dataset": "BTCH_rhythm_median_8lead", #ii_v1
    "epochs": 30, #30  100
    }
)
def to_np(x):
    return x.cpu().detach().numpy()

In [ ]:
from dataset_pretrain import pretrain_set as training_set
from dataset_pretrain import valid_set,test_set
print(training_set[0])

In [ ]:
from fcn import FCN
from  onedResnet_myopt import resnet18to34
import torch.nn as nn
class MyModelWithoutFC(nn.Module):#
    def __init__(self,out_ch):
        super(MyModelWithoutFC, self).__init__()
        self.out_ch = out_ch
        self.encoder = resnet18to34(num_classes=2)
        self.predictor = nn.Sequential(
            nn.Linear(self.out_ch, self.out_ch),
            nn.BatchNorm1d(self.out_ch),
            nn.ReLU(),
            nn.Linear(self.out_ch, 2)
        )
        self.proj_head = nn.Sequential(
            nn.Linear(self.out_ch, self.out_ch),
            nn.BatchNorm1d(self.out_ch),
            nn.ReLU(),
            nn.Linear(self.out_ch, self.out_ch)
        )

    def forward(self, x): 
        h = self.encoder(x)  
        out = self.proj_head(h)
        rr = self.predictor(h)
        return out, h, rr

In [ ]:
from NT_Xent import NT_Xent_loss
mse_loss = nn.MSELoss()
th.cuda.is_available()
from sklearn.metrics import roc_auc_score, confusion_matrix
def eval_fn(target,pred):
    cm = confusion_matrix(target, pred)
    acc = (cm[0][0]+cm[1][1])/cm.sum()
    print("acc",acc)
    print("cm",cm)
    sen,spe = cm[1][1]/cm[1].sum(),cm[0][0]/cm[0].sum()
    wandb.log({"Accuracy":acc,"Sensitivity":sen,"Specificity":spe})
    return acc

In [ ]:
#@title mixup model trainer per epoch
from transformations import *
import torch
from tqdm import tqdm

save_dir = "./Inter-Intraperiod-ECG/save_models/"  
# best_acc = 0
def train_mixup_model_epoch(model1,model2, training_set, test_set, optimizer1,optimizer2,  scheduler1,scheduler2,alpha, epochs):
    
    device = 'cuda' if th.cuda.is_available() else 'cpu'
    batch_size_tr = len(training_set)#len(training_set)#len(training_set)#len(training_set)#len(training_set.x)
    bs = 16
    best_acc = 0
    training_generator = DataLoader(training_set, batch_size=bs,
                                    shuffle=True, drop_last=True)
    for epoch in range(epochs):
        for rhythm,median,feat_rr,label in tqdm(training_generator,desc=f'Epoch {epoch + 1}/{epochs}'):
            
            rhythm,median,feat_rr = rhythm.to(device),median.to(device),feat_rr.to(device)
            model1.train()
            model2.train()
            optimizer1.zero_grad()
            optimizer2.zero_grad()
            
            p1,h1,rr = model1(rhythm)
            p2,h2 = model2(median)
           
            loss1 = NT_Xent_loss(p1,p2,temperature=0.5)
            loss2 = mse_loss(rr,feat_rr)
            loss = alpha*loss1 + (1-alpha)*loss2
            loss.backward()
            optimizer1.step()
            optimizer2.step()
            lr1 = optimizer1.param_groups[0]['lr']
            lr2 = optimizer2.param_groups[0]['lr']
            wandb.log({"loss_pred":loss2.item(),"loss_contrast":loss1.item(),"loss_all":loss.item(),
                       "lr1":lr1,"lr2":lr2})
        scheduler1.step()
        scheduler2.step()
        acc = test_model(model1, valid_set, test_set) 
        # wandb.log({"Accuracy":acc})
        if acc > best_acc:
            best_acc = acc
            th.save(model1.state_dict(), save_dir+"resnet18to34_best_acc.pth") 
            th.save(model2.state_dict(), save_dir+"fcn_best_acc.pth")
            print("update saving the model")
        if epoch == epochs - 1:
            th.save(model1.state_dict(), save_dir+"resnet18to34_final.pth")
            th.save(model2.state_dict(), save_dir+"fcn_final.pth")
            print("end")


In [ ]:

def test_model(model, valid_set, test_set):
    out_ch = 512 #128
    batch_size = 200
    model.eval()

    N_tr = len(valid_set)#len(training_set.x)
    N_te = len(test_set) #len(test_set.x)

    valid_generator = DataLoader(valid_set, batch_size=batch_size,
                                    shuffle=True, drop_last=False)
    test_generator = DataLoader(test_set, batch_size= batch_size,
                                    shuffle=True, drop_last=False)

    H_tr = th.zeros((N_tr, out_ch))
    y_tr = th.zeros((N_tr), dtype=th.long)

    H_te = th.zeros((N_te, out_ch))
    y_te = th.zeros((N_te), dtype=th.long)

    for batch_idx, (x_tr, _, _, y_tr_i) in enumerate(valid_generator):
        with torch.no_grad():
            x_tr = x_tr.to("cuda")
            _, H_tr_i, _ = model(x_tr)
            H_tr[batch_idx * batch_size: (batch_idx + 1) * batch_size] = H_tr_i
            y_tr[batch_idx * batch_size: (batch_idx + 1) * batch_size] = y_tr_i

    H_tr = to_np(nn.functional.normalize(H_tr))
    y_tr = to_np(y_tr)

    for batch_idx, (x_te, _, _, y_te_i) in enumerate(test_generator):
        with torch.no_grad():
            x_te = x_te.to("cuda")
            _, H_te_i, _ = model(x_te)
            H_te[batch_idx * batch_size: (batch_idx + 1) * batch_size] = H_te_i
            y_te[batch_idx * batch_size: (batch_idx + 1) * batch_size] = y_te_i
    

    H_te = to_np(nn.functional.normalize(H_te))
    y_te = to_np(y_te)

    clf = KNeighborsClassifier(n_neighbors=1).fit(H_tr, y_tr) #对于FCN扩大k1to3 能提点

    pred = clf.predict(H_te)#weighted KNN
    acc = eval_fn(y_te,pred)
    return acc


In [ ]:

import warnings
warnings.filterwarnings("ignore")

device = 'cuda' if th.cuda.is_available() else 'cpu'
epochs = 30 #100
alpha = 0.5 #0.5

model1 = MyModelWithoutFC(out_ch=512).to(device) #512+2
model2 = FCN(n_in=2).to(device)  #n_in=8
optimizer1 = th.optim.Adam(model1.parameters(),lr=0.0003)
optimizer2 = th.optim.Adam(model2.parameters(),lr=0.0003)
scheduler1 = torch.optim.lr_scheduler.StepLR(optimizer1,step_size=3,gamma=0.9)
scheduler2 = torch.optim.lr_scheduler.StepLR(optimizer2,step_size=3,gamma=0.9)
train_mixup_model_epoch(model1,model2, training_set, test_set,
                                              optimizer1,optimizer2, scheduler1,scheduler2,alpha, epochs)
